In [213]:
#udemy a-crash-course-in-pyspark/
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as sqlfunc
import numpy as np

In [ ]:
#avg pay of woman and man
#avg pay of woman and man by jobtitle
#which city has highest avg salary

In [20]:
spark = SparkSession.builder.appName('mymlproject').getOrCreate()

In [21]:
spark

In [248]:
#df = spark.read.format('csv').option('header','true').load('data/original.csv')
df = spark.read.csv('data/original.csv',header=True)

In [251]:
df.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [262]:
from pyspark.sql.types import *
schema = StructType([
    StructField('id',IntegerType()),
    StructField('first_name',StringType()),
    StructField('last_name',StringType()),
    StructField('gender',StringType()),
    StructField('City',StringType()),
    StructField('JobTitle',StringType()),
    StructField('Salary',StringType()),
    StructField('Latitude',FloatType()),
    StructField('Longitude',FloatType())
])
df = spark.read.csv('data/original.csv',header=True, schema=schema)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [24]:
df_clean = df.withColumn('clean_city',
                         when(df.City.isNull(),'Unknown')
                         .otherwise(df.City))

In [257]:
df_clean.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|         clean_lat|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875| 50.57740783691406|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|48.823158264160156|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|

In [27]:
df_clean = df_clean.filter(df_clean.JobTitle.isNotNull())

In [37]:
df_clean.show(10,True)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
|  8|   Goddart|     Flear|  Mal

In [39]:
#if salary is null then replace with avg salary
#remove $ in salary
df_clean = df_clean.withColumn('clean_salary',
                              df_clean.Salary.substr(2,100).cast('float'))

In [42]:
df_clean.show(1,True,True)

-RECORD 0---------------------------
 id           | 1                   
 first_name   | Melinde             
 last_name    | Shilburne           
 gender       | Female              
 City         | Nowa Ruda           
 JobTitle     | Assistant Professor 
 Salary       | $57438.18           
 Latitude     | 50.5774075          
 Longitude    | 16.4967184          
 clean_city   | Nowa Ruda           
 clean_salary | 57438.18            
only showing top 1 row



In [58]:
sal_mean = df_clean.groupby().avg('clean_salary').take(1)[0][0]
sal_mean

55516.32088199837

In [61]:
from pyspark.sql.functions import lit
df_clean = df_clean.withColumn('new_salary'
                               ,when(df_clean.clean_salary.isNull(),lit(sal_mean))
                               .otherwise(df_clean.clean_salary))

In [63]:
df_clean.show(10)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|      new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|   57438.1796875|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6015625|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23046875|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 3

In [196]:
#populate latitude and longitude with median values
lat_median = df_clean.filter(df_clean.Latitude.isNotNull()).select('Latitude')
lat_median = lat_median.withColumn('float_lat',lat_median.Latitude.cast('float')).select('float_lat')
med = np.median(lat_median.collect())
med

31.93397331237793

In [208]:
df_clean = df_clean.withColumn('clean_lat',
                              when(df_clean.Latitude.isNull(),lit(med)).
                              otherwise(df_clean.Latitude.cast('float')))

In [220]:
#avg pay of woman and man
genders = df_clean.groupby('gender').agg(sqlfunc.avg('clean_salary').alias('AvgSalary'))

In [221]:
genders.show()

+------+------------------+
|gender|         AvgSalary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [ ]:
#avg pay of woman and man by jobtitle
salaries = df_clean.withColumn('female_salary',when(df_clean.gender=='Female',df_clean.clean_salary)
                               .otherwise(lit(0)))
salaries = salaries.withColumn('male_salary',when(salaries.gender=='Male',salaries.clean_salary)
                               .otherwise(lit(0)))
salaries.show()

In [ ]:
final_salary = salaries.groupby('JobTitle').agg(sqlfunc.avg('female_salary').alias('final_female_salary'),
               sqlfunc.avg('male_salary').alias('final_male_salary'))
final_salary.show(100,False)

In [237]:
delta = final_salary.withColumn('delta',
                               final_salary.final_female_salary - final_salary.final_male_salary)
delta.show()

+--------------------+-------------------+------------------+-------------------+
|            JobTitle|final_female_salary| final_male_salary|              delta|
+--------------------+-------------------+------------------+-------------------+
|Systems Administr...|    50590.474609375|  15540.9501953125|   35049.5244140625|
|   Media Manager III| 29586.436197916668|17381.920572916668|       12204.515625|
|  Recruiting Manager| 34848.452473958336|  26383.4951171875|  8464.957356770836|
|       Geologist III|       31749.046875|    12830.75390625|     18918.29296875|
|        Geologist II|                0.0|   43293.865234375|   -43293.865234375|
|Database Administ...|                0.0|     52018.4609375|     -52018.4609375|
|   Financial Analyst|    23353.776953125|       39606.05625|   -16252.279296875|
|  Analyst Programmer|   16406.1287109375|  21042.9634765625| -4636.834765625001|
|Software Engineer II|                0.0|      74782.640625|      -74782.640625|
|       Accounta

In [245]:
#which city has highest avg salary
city_salary = salaries.groupby('City').agg(sqlfunc.avg('clean_salary').alias('city_avg'))
city_salary = city_salary.sort(col('city_avg').desc())
city_salary.show()

+-----------------+-------------+
|             City|     city_avg|
+-----------------+-------------+
|        Mesopotam|  99948.28125|
|       Zhongcheng| 99942.921875|
|           Caxias|99786.3984375|
|      Karangtawar|99638.9921875|
|        Itabaiana|  99502.15625|
|           Pasian|  99421.34375|
|           Webuye| 99368.546875|
|      Yuktae-dong| 99250.828125|
|           Zinder|  99222.84375|
|   Timiryazevskiy|   99142.9375|
|        Sawahbaru|99013.7109375|
|          Madimba|98737.8671875|
|         Huangshi|  98690.34375|
|          Gharyan|   98679.3125|
|         Yŏnan-ŭp| 98628.609375|
|     Wringinputih|98603.8203125|
|Monte da Boavista|  98586.71875|
|          Klukeng|98439.4921875|
|         Murmashi|  98226.15625|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows



In [264]:
df = spark.read.csv('data/original.csv',header=True, schema=schema)
df.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'float'),
 ('Longitude', 'float')]

In [265]:
df.head(1)

[Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude=50.57740783691406, Longitude=16.49671745300293)]

In [269]:
x = df.first()
x

Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude=50.57740783691406, Longitude=16.49671745300293)

In [271]:
df.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|          Latitude|        Longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|               999|             1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null| 25.43151724702484|43.33756460386515|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|     null|24.579082550156635| 69.4206453674681|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|         -54.28115|       -123.04196|

In [273]:
df.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [275]:
df.count()

1000

In [276]:
df.distinct().count()

1000

In [277]:
#handling null & duplicates
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [278]:
#drop rows with any col Na
df_dropped = df.na.drop()
df_dropped.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Manage...|$73697.10| 32.027935| 106.65711|
| 10|      Bran|   Trahear|  Male|       Arbeláez|Mechanical Sys

In [281]:
df_dropped_jobTitle = df.filter(df.JobTitle.isNotNull())
df_dropped_jobTitle.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  8|   Goddart|     Flear|  Male|      Trélissac|Desktop Support T...|$46116.36| 45.190517| 0.7423124|
|  9|      Roth|O'Cannavan|  Male|         Heitan|VP Product Man

In [283]:
df_dupdropped = df.dropDuplicates()
df_dupdropped.show()

+---+----------+-------------+------+--------------------+--------------------+---------+----------+----------+
| id|first_name|    last_name|gender|                City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+-------------+------+--------------------+--------------------+---------+----------+----------+
|372|     Lyman|      Burfitt|  Male|             Guiping|Community Outreac...|$28755.53| 23.394325| 110.07938|
|391|     Verge|     Hefferan|  Male|          Cocachacra|Community Outreac...|$90391.71|-17.091843| -71.77114|
|428|   Pernell|      Fossitt|  Male|           Wangchang|    Graphic Designer|$36927.53| 40.401047|  117.9989|
|526|  Garfield|    Benadette|  Male|              Shiren|   Marketing Manager|$56867.78| 28.651703| 117.90413|
|731|     Legra|        Manns|Female|               Nîmes|  Research Associate|$42246.87| 43.844727| 4.3520436|
|809|       Jed|       Shires|  Male|            Goubétto|     Design Engineer|$64130.79| 11.423197| 43.

In [284]:
#selecting data
df_new = df.select(['first_name','last_name'])
df_new.show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|   Melinde| Shilburne|
|  Kimberly|Von Welden|
|    Alvera|  Di Boldi|
|   Shannon| O'Griffin|
|  Sherwood|   Macieja|
|     Maris|      Folk|
|     Masha|    Divers|
|   Goddart|     Flear|
|      Roth|O'Cannavan|
|      Bran|   Trahear|
|    Kylynn|   Lockart|
|       Rey|    Meharg|
|      Kerr|    Braden|
|    Mickie| Whanstall|
|    Kaspar|     Pally|
|    Norbie|    Gwyllt|
|    Claude|    Briant|
|     Thain|    Habbon|
|  Tiffanie|  Pattison|
|    Ettore|  Gerriets|
+----------+----------+
only showing top 20 rows



In [285]:
df_ren = df_new.withColumnRenamed('first_name','fn')
df_ren.show()

+--------+----------+
|      fn| last_name|
+--------+----------+
| Melinde| Shilburne|
|Kimberly|Von Welden|
|  Alvera|  Di Boldi|
| Shannon| O'Griffin|
|Sherwood|   Macieja|
|   Maris|      Folk|
|   Masha|    Divers|
| Goddart|     Flear|
|    Roth|O'Cannavan|
|    Bran|   Trahear|
|  Kylynn|   Lockart|
|     Rey|    Meharg|
|    Kerr|    Braden|
|  Mickie| Whanstall|
|  Kaspar|     Pally|
|  Norbie|    Gwyllt|
|  Claude|    Briant|
|   Thain|    Habbon|
|Tiffanie|  Pattison|
|  Ettore|  Gerriets|
+--------+----------+
only showing top 20 rows



In [286]:
#filter
df_filter = df_ren.filter(df_ren.fn=='Alvera')
df_filter.show()

+------+---------+
|    fn|last_name|
+------+---------+
|Alvera| Di Boldi|
+------+---------+



In [296]:
df_filter = df_ren.filter(df_ren.fn.like('%lvera'))
df_filter.show()

+------+---------+
|    fn|last_name|
+------+---------+
|Alvera| Di Boldi|
+------+---------+



In [298]:
df_filter = df_ren.filter((df_ren.fn.endswith('lvera')))
df_filter.show()

+------+---------+
|    fn|last_name|
+------+---------+
|Alvera| Di Boldi|
+------+---------+



In [301]:
df_filter = df.filter((df.id.between(1,4)))
df_filter.show()

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+



In [304]:
df_filter = df.filter((df.first_name.isin('Melinde','Alvera')))
df_filter.show()

+---+----------+---------+------+---------+-------------------+---------+---------+----------+
| id|first_name|last_name|gender|     City|           JobTitle|   Salary| Latitude| Longitude|
+---+----------+---------+------+---------+-------------------+---------+---------+----------+
|  1|   Melinde|Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.577408| 16.496717|
|  3|    Alvera| Di Boldi|Female|     null|               null|$57576.52|39.994747|116.339775|
+---+----------+---------+------+---------+-------------------+---------+---------+----------+



In [306]:
df_filter = df.select(df.first_name,df.first_name.substr(1,4).alias('f_trim'))
df_filter.show()

+----------+------+
|first_name|f_trim|
+----------+------+
|   Melinde|  Meli|
|  Kimberly|  Kimb|
|    Alvera|  Alve|
|   Shannon|  Shan|
|  Sherwood|  Sher|
|     Maris|  Mari|
|     Masha|  Mash|
|   Goddart|  Godd|
|      Roth|  Roth|
|      Bran|  Bran|
|    Kylynn|  Kyly|
|       Rey|   Rey|
|      Kerr|  Kerr|
|    Mickie|  Mick|
|    Kaspar|  Kasp|
|    Norbie|  Norb|
|    Claude|  Clau|
|     Thain|  Thai|
|  Tiffanie|  Tiff|
|    Ettore|  Etto|
+----------+------+
only showing top 20 rows



In [308]:
df_filter = df.filter((df.first_name.isin('Melinde','Alvera')) & (df.City.isin('Nowa Ruda')))
df_filter.show()

+---+----------+---------+------+---------+-------------------+---------+---------+---------+
| id|first_name|last_name|gender|     City|           JobTitle|   Salary| Latitude|Longitude|
+---+----------+---------+------+---------+-------------------+---------+---------+---------+
|  1|   Melinde|Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.577408|16.496717|
+---+----------+---------+------+---------+-------------------+---------+---------+---------+



In [309]:
df_filter = df.filter((df.first_name.isin('Melinde','Alvera')) | (df.City.isin('Nowa Ruda')))
df_filter.show()

+---+----------+---------+------+---------+-------------------+---------+---------+----------+
| id|first_name|last_name|gender|     City|           JobTitle|   Salary| Latitude| Longitude|
+---+----------+---------+------+---------+-------------------+---------+---------+----------+
|  1|   Melinde|Shilburne|Female|Nowa Ruda|Assistant Professor|$57438.18|50.577408| 16.496717|
|  3|    Alvera| Di Boldi|Female|     null|               null|$57576.52|39.994747|116.339775|
+---+----------+---------+------+---------+-------------------+---------+---------+----------+



In [312]:
#running sql on dataframe
#create a temp table from df so that on that table we run sql 
df.registerTempTable('original')

In [313]:
query1 = spark.sql('select * from original')
query1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

In [318]:
query2 = spark.sql('select concat(first_name," ",last_name )as full_name from original where gender="Female"')
query2.show()
#so you use " inside sql query

+-------------------+
|          full_name|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|         Maris Folk|
|       Masha Divers|
|     Kylynn Lockart|
|         Rey Meharg|
|      Claude Briant|
|  Tiffanie Pattison|
|    Lurleen Janczak|
|      Nichol Holtum|
|       Shaun Bridle|
|     Leandra Anfrey|
|    Jaquelyn Hazard|
|  Prudence Honacker|
|       Cherey Liger|
|          Neda Krop|
|    Barbi Fattorini|
|   Lonnie Townshend|
|    Valida Salzberg|
+-------------------+
only showing top 20 rows



In [322]:
#adding calculated columlns
df_clean = df.withColumn('clean_salary',
                        df.Salary.substr(2,100).cast('float'))
df_clean.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|     62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|    57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil En

In [323]:
df_clean.registerTempTable('original_clean')

In [324]:
check = spark.sql('select * from original_clean')
check.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|     62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|    57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil En

In [327]:
df_Cust = df.withColumn('Is_female',when(df.gender == 'Female','Yes').otherwise('No'))
df_Cust.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+---------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|Is_female|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+---------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|      Yes|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|      Yes|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|      Yes|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|       No|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|       No|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426

In [332]:
#data aggregation
df_grp = df_clean.groupby('gender').agg(sqlfunc.sum('clean_salary').alias('sum_salary'),
                                       sqlfunc.avg('clean_salary').alias('avg_salary'),
                                       sqlfunc.min('clean_salary').alias('min_salary'),
                                       sqlfunc.max('clean_salary').alias('max_salary'))
df_grp.show()

+------+--------------------+-----------------+----------+----------+
|gender|          sum_salary|       avg_salary|min_salary|max_salary|
+------+--------------------+-----------------+----------+----------+
|Female|2.7364519950195312E7|55618.94298820185|  10616.44|  99948.28|
|  Male|2.8123435678710938E7|55361.09385573019|  10101.92|  99942.92|
+------+--------------------+-----------------+----------+----------+



In [333]:
df_grp = df_clean.groupby(['gender','city']).agg(sqlfunc.sum('clean_salary').alias('sum_salary'),
                                       sqlfunc.avg('clean_salary').alias('avg_salary'),
                                       sqlfunc.min('clean_salary').alias('min_salary'),
                                       sqlfunc.max('clean_salary').alias('max_salary'))
df_grp.show()

+------+-----------------+----------------+----------------+----------+----------+
|gender|             city|      sum_salary|      avg_salary|min_salary|max_salary|
+------+-----------------+----------------+----------------+----------+----------+
|Female|           Dachun| 25090.869140625| 25090.869140625|  25090.87|  25090.87|
|Female|      Trollhättan|106623.369140625|53311.6845703125|  26830.47|   79792.9|
|  Male|          Wenshao| 18941.509765625| 18941.509765625|  18941.51|  18941.51|
|Female|            Lanas| 13765.900390625| 13765.900390625|   13765.9|   13765.9|
|  Male|            Mörön|    77940.078125|    77940.078125|  77940.08|  77940.08|
|Female|             Same|   73369.7265625|   73369.7265625|  73369.73|  73369.73|
|Female|          Sawahan|  24608.83984375|  24608.83984375|  24608.84|  24608.84|
|  Male|Monte da Boavista|     98586.71875|     98586.71875|  98586.72|  98586.72|
|Female|         Nusajaya|    71637.921875|    71637.921875|  71637.92|  71637.92|
|Fem

In [334]:
#writing df to csv
df_grp.write.csv('grp.csv')
#writes data as partitioned one